In [37]:
import json
import fileinput
import requests
from requests.exceptions import Timeout
import re
import os
import time

# helper function to convert our text files to json
def text2json(text):
    fin = fileinput.input(text, inplace=1)
    for line in fin:
        if fin.isfirstline():
            print ("[" + line)
        else:
            print ("," + line)
    fin.close()
    
    f = open(text,"a")
    f.write("]")
    f.close()
    
# load json from textfile
def loaddata(text): 
    # if json.load fails, format
    try:
        f = open(text, 'r')
        data = json.load(f)
        f.close()
        return data 
    except:
        print("Formatting " + text + "...")
        text2json(text)
        return loaddata(text)

# load title from url
def gettitle(url):
    try:
        r = requests.get(url, timeout=1.5)
        if r:
            return re.findall("(?<=<title>)\s*(.*)\s*(?=<\/title>)", r.text)
        else:
            return ''
    except Timeout: 
        print("Request timed out")
        return ''
    except Exception as e: 
        print("Exception: ", str(e))
        return ''
        
    # return text between <title> tags in html 

# add url title to ['title'] attribute, textfile to new file
def addtitle(textfile, outdir, counter):
    data = loaddata(textfile)
    try:
        print("Starting to convert file " + textfile)
        for tweet in data:
            titles = []
            for url in tweet['tweet'][0]['entities']['urls']:
                title = gettitle(url['expanded_url'])
                titles.append(title)
            tweet['titles'] = titles
        outfile = outdir + "t-" + str(counter) + ".json"
        with open(outfile, 'w') as file:
             file.write(json.dumps(data)) # use `json.loads` to do the reverse
        print("Finished converting file " + textfile)
    except Exception as e:
        print("Exception thrown: " + str(e))
    

In [38]:
import os

# set up dir
#os.chdir("tweetsE")
directory = os.getcwd()
files = os.listdir(directory)

counter = 10
odir = "/Users/airwi/Desktop/twitter-scraper/tweetsETitles/"

print(directory)

for f in files:
    filename, file_extension = os.path.splitext(f)
    if(file_extension == '.txt'):
        addtitle(f, odir, counter)
        counter += 1

C:\Users\airwi\Desktop\twitter-scraper\tweetsE
Formatting t2-25.txt...


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 't2-25.txt' -> 't2-25.txt.bak'